### **A Tool for Evaluating and Analysing the Convenience of Property Location Based on 15-minute City Paradigm**

Xianlai Yin, MSc Urban Spatial Science, CASA, UCL

---


***Environment***

- ArcGIS Pro
  - install threadpoolctl
  - update scipy
  - update pyarrow

***Coordinate Systm***

- British National Grid (EPSG:27700)

***Input File Require*** *(put all these files in a same folder named "input" in workspace)*

- pois.shp
- landuse.shp
- area.shp
- meanprice.shp
- meanprice_idw.tif
- population.tif
- walkdistance.csv
- areacodelist.csv
- census (folder)
  - RM200-Sex-By-Single-Year-Of-Age-(Detailed)-2021-lsoa-ONS.csv
  - TS004-Country-Of-Birth-2021-lsoa-ONS.csv
  - TS016-Length-Of-Residence-2021-lsoa-ONS.csv
  - TS017-Household-Size-2021-lsoa-ONS.csv
  - TS021-Ethnic-Group-2021-lsoa-ONS.csv
  - TS030-Religion-2021-lsoa-ONS.csv
  - TS045-Car-Or-Van-Availability-2021-lsoa-ONS.csv
  - TS051-Number-Of-Rooms-2021-lsoa-ONS.csv
  - TS058-Distance-Travelled-To-Work-2021-lsoa-ONS.csv
  - TS066-Economic-Activity-Status-2021-lsoa-ONS.csv


***Example Data Area***

- Greater London

In [ ]:
### test parameters
#folder="D:\\UCL_CODE\\data\\workspace"
#area_type="borough"
#with open(f"{folder}\\input\\areacodelist.csv", "r", encoding="utf-8") as file:
    #header = file.readline()
    #areacodes = header.split(",").index("LAD23CD")
    #for line in file:
        #values = line.strip().split(",")
        #area = values[areacodes]

#### analysispreparation.py

***Parameters***

|Name|Data Type|Type|Direction|
|---|---|---|---|
|folder|Folder|Required|Input|
|area|String|Required|Input|
|area_type|String|Required|Input|
|success|Boolean|Derived|Output|

In [ ]:
### import
import arcpy
from arcpy.sa import *
from sys import argv
import os


### input parameter
folder=arcpy.GetParameterAsText(0)
area=arcpy.GetParameterAsText(1)
area_type=arcpy.GetParameterAsText(2)


### environment
arcpy.env.overwriteOutput = True
arcpy.env.workspace = folder


### define functions
def datapreparation(folder, area, area_type):
    # define the function to find the raw data
    def inputraw(folder):
        area_raw = f"{folder}\\input\\area.shp"
        population_raw = f"{folder}\\input\\population.tif"
        pois_raw = f"{folder}\\input\\pois.shp"
        roads_raw = f"{folder}\\input\\roads.shp"
        landuse_raw = f"{folder}\\input\\landuse.shp"
        return area_raw, population_raw, pois_raw, roads_raw, landuse_raw
    area_raw, population_raw, pois_raw, roads_raw, landuse_raw = inputraw(folder)
    
    # define the function to select the data
    def dataselect(folder, area, area_type, area_raw, population_raw, pois_raw, roads_raw, landuse_raw, _area_parks=r"{folder}\{area}\{area}_pois.gdb\{area}_parks", _area_roads=r"{folder}\{area}\{area}_roads.gdb\{area}_roads", _area_population=r"{folder}\{area}\{area}_population.gdb\{area}_population"):
        # create gdb
        _folder_ = f"{folder}"
        # Process: Create Folder (Create Folder) (management)
        _area_folder = arcpy.management.CreateFolder(out_folder_path=_folder_, out_name=f"{area}")[0]
        # Process: Create File Geodatabase boundaries (Create File Geodatabase) (management)
        _area_boundaries_gdb = arcpy.management.CreateFileGDB(out_folder_path=_area_folder, out_name=f"{area}_boundaries")[0]
        # Process: Create File Geodatabase pois (Create File Geodatabase) (management)
        _area_pois_gdb = arcpy.management.CreateFileGDB(out_folder_path=_area_folder, out_name=f"{area}_pois")[0]
        # Process: Create File Geodatabase others (Create File Geodatabase) (management)
        _area_others_gdb = arcpy.management.CreateFileGDB(out_folder_path=_area_folder, out_name=f"{area}_others")[0]
        # Process: Create File Geodatabase population (Create File Geodatabase) (management)
        _area_population_gdb = arcpy.management.CreateFileGDB(out_folder_path=_area_folder, out_name=f"{area}_population")[0]
        # Process: Create File Geodatabase roads (Create File Geodatabase) (management)
        #_area_roads_gdb = arcpy.management.CreateFileGDB(out_folder_path=_area_folder, out_name=f"{area}_roads")[0]
        
        # create areas
        # define the function to judge the area type
        def boroughorward(area_type):
            is_borough = area_type in ["borough", "Borough", "LAD"]
            is_ward = not is_borough
            return is_borough, is_ward
        # Process: If Value Is (If Value Is)
        borough, ward = boroughorward(area_type)
        # Process: select ward (Feature Class To Feature Class) (conversion)
        if ward:
            _area_ward = arcpy.conversion.FeatureClassToFeatureClass(in_features=area_raw.__str__().format(**globals()), out_path=_area_boundaries_gdb, out_name=f"{area}_ward", where_clause=f"WD23CD = '{area}'", field_mapping="WD23CD \"WD23CD\" true true false 9 Text 0 0,First,#,{area_raw},WD23CD,0,9;WD23NM \"WD23NM\" true true false 53 Text 0 0,First,#,{area_raw},WD23NM,0,53;LAD23CD \"LAD23CD\" true true false 9 Text 0 0,First,#,{area_raw},LAD23CD,0,9;LAD23NM \"LAD23NM\" true true false 36 Text 0 0,First,#,{area_raw},LAD23NM,0,36")[0]
            analysis_area = _area_ward
        # Process: select borough (Feature Class To Feature Class) (conversion)
        if borough:
            _area_borough = arcpy.conversion.FeatureClassToFeatureClass(in_features=area_raw.__str__().format(**globals()), out_path=_area_boundaries_gdb, out_name=f"{area}_borough", where_clause=f"LAD23CD = '{area}'", field_mapping="WD23CD \"WD23CD\" true true false 9 Text 0 0,First,#,{area_raw},WD23CD,0,9;WD23NM \"WD23NM\" true true false 53 Text 0 0,First,#,{area_raw},WD23NM,0,53;LAD23CD \"LAD23CD\" true true false 9 Text 0 0,First,#,{area_raw},LAD23CD,0,9;LAD23NM \"LAD23NM\" true true false 36 Text 0 0,First,#,{area_raw},LAD23NM,0,36")[0]
            analysis_area = _area_borough
        # Process: Dissolve (Dissolve) (management)
        _area_ = fr"{folder}\{area}\{area}_others.gdb\{area}_boundary"
        arcpy.management.Dissolve(in_features=analysis_area, out_feature_class=_area_)
        # Process: Buffer (Buffer) (analysis)
        _area_Buffer = fr"{folder}\{area}\{area}_others.gdb\{area}_Buffer"
        arcpy.analysis.Buffer(in_features=_area_, out_feature_class=_area_Buffer, buffer_distance_or_field="1500 Meters", line_end_type="ROUND", dissolve_option="ALL")
        # Process: Intersect pois (Intersect) (analysis)
        pois_area_ = fr"{folder}\{area}\{area}_others.gdb\pois_{area}"
        arcpy.analysis.Intersect(in_features=[[_area_Buffer, ""], [pois_raw, ""]], out_feature_class=pois_area_, join_attributes="NO_FID")
        
        # pois
        # Process: create health (Feature Class To Feature Class) (conversion)
        _area_health = arcpy.conversion.FeatureClassToFeatureClass(in_features=pois_area_, out_path=_area_pois_gdb, out_name=f"{area}_health", where_clause="code >= 2101 And code <= 2121 Or code = 2013", field_mapping="code \"code\" true true false 4 Short 0 4,First,#,{pois_raw},code,-1,-1;fclass \"fclass\" true true false 28 Text 0 0,First,#,{pois_raw},fclass,0,28")[0]
        # Process: create leisure (Feature Class To Feature Class) (conversion)
        _area_leisure = arcpy.conversion.FeatureClassToFeatureClass(in_features=pois_area_, out_path=_area_pois_gdb, out_name=f"{area}_leisure", where_clause="code >= 2201 And code <= 2203 Or code = 2721 Or code = 2722 Or code = 2725 Or code = 2743 Or code = 2744", field_mapping="code \"code\" true true false 4 Short 0 4,First,#,{pois_raw},code,-1,-1;fclass \"fclass\" true true false 28 Text 0 0,First,#,{pois_raw},fclass,0,28")[0]
        # Process: create sports (Feature Class To Feature Class) (conversion)
        _area_sports = arcpy.conversion.FeatureClassToFeatureClass(in_features=pois_area_, out_path=_area_pois_gdb, out_name=f"{area}_sports", where_clause="(code >= 2251 And code <= 2257)", field_mapping="code \"code\" true true false 4 Short 0 4,First,#,{pois_raw},code,-1,-1;fclass \"fclass\" true true false 28 Text 0 0,First,#,{pois_raw},fclass,0,28")[0]
        # Process: create catering (Feature Class To Feature Class) (conversion)
        _area_catering = arcpy.conversion.FeatureClassToFeatureClass(in_features=pois_area_, out_path=_area_pois_gdb, out_name=f"{area}_catering", where_clause="code >= 2301 And code <= 2307 Or code = 2502", field_mapping="code \"code\" true true false 4 Short 0 4,First,#,{pois_raw},code,-1,-1;fclass \"fclass\" true true false 28 Text 0 0,First,#,{pois_raw},fclass,0,28")[0]
        # Process: create shopping (Feature Class To Feature Class) (conversion)
        _area_shopping = arcpy.conversion.FeatureClassToFeatureClass(in_features=pois_area_, out_path=_area_pois_gdb, out_name=f"{area}_shopping", where_clause="(code >= 2512 And code <= 2530) Or code >= 2542 And code <= 2561 Or code = 2568", field_mapping="code \"code\" true true false 4 Short 0 4,First,#,{pois_raw},code,-1,-1;fclass \"fclass\" true true false 28 Text 0 0,First,#,{pois_raw},fclass,0,28")[0]
        # Process: create supermarket (Feature Class To Feature Class) (conversion)
        _area_supermarket = arcpy.conversion.FeatureClassToFeatureClass(in_features=pois_area_, out_path=_area_pois_gdb, out_name=f"{area}_supermarket", where_clause="(code >= 2503 And code <= 2511) Or (code = 2501) Or code = 2016", field_mapping="code \"code\" true true false 4 Short 0 4,First,#,{pois_raw},code,-1,-1;fclass \"fclass\" true true false 28 Text 0 0,First,#,{pois_raw},fclass,0,28")[0]
        # Process: create financial (Feature Class To Feature Class) (conversion)
        _area_financial = arcpy.conversion.FeatureClassToFeatureClass(in_features=pois_area_, out_path=_area_pois_gdb, out_name=f"{area}_financial", where_clause="(code >= 2601 And code <= 2602)", field_mapping="code \"code\" true true false 4 Short 0 4,First,#,{pois_raw},code,-1,-1;fclass \"fclass\" true true false 28 Text 0 0,First,#,{pois_raw},fclass,0,28")[0]
        # Process: create education (Feature Class To Feature Class) (conversion)
        _area_education = arcpy.conversion.FeatureClassToFeatureClass(in_features=pois_area_, out_path=_area_pois_gdb, out_name=f"{area}_education", where_clause="code >= 2082 And code <= 2083", field_mapping="code \"code\" true true false 4 Short 0 4,First,#,{pois_raw},code,-1,-1;fclass \"fclass\" true true false 28 Text 0 0,First,#,{pois_raw},fclass,0,28")[0]
        # Process: create facilities (Feature Class To Feature Class) (conversion)
        _area_facilities = arcpy.conversion.FeatureClassToFeatureClass(in_features=pois_area_, out_path=_area_pois_gdb, out_name=f"{area}_facilities", where_clause="code = 2001 Or code = 2002 Or code = 2007 Or code = 2012 Or code = 2003", field_mapping="code \"code\" true true false 4 Short 0 4,First,#,{pois_raw},code,-1,-1;fclass \"fclass\" true true false 28 Text 0 0,First,#,{pois_raw},fclass,0,28")[0]
        # Process: create worship (Feature Class To Feature Class) (conversion)
        _area_worship = arcpy.conversion.FeatureClassToFeatureClass(in_features=pois_area_, out_path=_area_pois_gdb, out_name=f"{area}_worship", where_clause="(code >= 3100 And code <= 3800)", field_mapping="osm_id \"osm_id\" true true false 12 Text 0 0,First,#,{pois_raw},osm_id,0,12;code \"code\" true true false 4 Short 0 4,First,#,{pois_raw},code,-1,-1;fclass \"fclass\" true true false 28 Text 0 0,First,#,{pois_raw},fclass,0,28")[0]
        # Process: create transport (Feature Class To Feature Class) (conversion)
        _area_transport = arcpy.conversion.FeatureClassToFeatureClass(in_features=pois_area_, out_path=_area_pois_gdb, out_name=f"{area}_transport", where_clause="code >= 5601 And code <= 5641", field_mapping="osm_id \"osm_id\" true true false 12 Text 0 0,First,#,{pois_raw},osm_id,0,12;code \"code\" true true false 4 Short 0 4,First,#,{pois_raw},code,-1,-1;fclass \"fclass\" true true false 28 Text 0 0,First,#,{pois_raw},fclass,0,28")[0]
        
        # parks
        # Process: create greenspace (Feature Class To Feature Class) (conversion)
        greenspace = arcpy.conversion.FeatureClassToFeatureClass(in_features=landuse_raw.__str__().format(**globals()), out_path=_area_others_gdb, out_name="greenspace", where_clause="code_2018 = '14100' Or code_2018 = '31000' Or code_2018 = '23000' Or code_2018 = '31000' Or code_2018 = '32000' Or code_2018 = '14200'", field_mapping="code_2018 \"code_2018\" true true false 5 Text 0 0,First,#,{landuse_raw},code_2018,0,5")[0]
        # Process: Dissolve greenspace (Dissolve) (management)
        greenspace_dissolve = fr"{folder}\{area}\{area}_others.gdb\greenspace_Dissolve"
        arcpy.management.Dissolve(in_features=greenspace, out_feature_class=greenspace_dissolve)
        # Process: Intersect greenspace (Intersect) (analysis)
        _area_greenspcae = fr"{folder}\{area}\{area}_others.gdb\greenspace_{area}"
        arcpy.analysis.Intersect(in_features=[[greenspace_dissolve, ""], [_area_Buffer, ""]], out_feature_class=_area_greenspcae, join_attributes="ONLY_FID")
        # Process: Polygon To Line (Polygon To Line) (management)
        greenspace_line = fr"{folder}\{area}\{area}_others.gdb\greenspace_line"
        arcpy.management.PolygonToLine(in_features=_area_greenspcae, out_feature_class=greenspace_line)
        # Process: Generate Points Along Lines (Generate Points Along Lines) (management)
        greenspace_point = fr"{folder}\{area}\{area}_others.gdb\greenspace_point"
        arcpy.management.GeneratePointsAlongLines(Input_Features=greenspace_line, Output_Feature_Class=greenspace_point, Point_Placement="DISTANCE", Distance="200 Meters", Include_End_Points="NO_END_POINTS")
        # Process: create greenspcae_pointtojoin (Feature Class To Feature Class) (conversion)
        greenspace_pointtojoin = arcpy.conversion.FeatureClassToFeatureClass(in_features=greenspace_point, out_path=_area_others_gdb, out_name="greenspace_pointtojoin", field_mapping="Shape_length \"Shape_length\" true true false 0 Double 0 0,First,#,greenspace_point,Shape_Length,-1,-1;LEFT_FID \"LEFT_FID\" true true false 0 Long 0 0,First,#,greenspace_point,LEFT_FID,-1,-1;RIGHT_FID \"RIGHT_FID\" true true false 0 Long 0 0,First,#,greenspace_point,RIGHT_FID,-1,-1;ORIG_FID \"ORIG_FID\" true true false 0 Short 0 0,First,#,greenspace_point,ORIG_FID,-1,-1")[0]
        # Process: Add Fields (multiple) (Add Fields (multiple)) (management)
        greenspace_pointtojoin2 = arcpy.management.AddFields(in_table=greenspace_pointtojoin, field_description=[["code", "SHORT", "", "", "2207", ""], ["fclass", "TEXT", "", "255", "greenspace", ""]])[0]
        # Process: create parks_pointtojoin (Feature Class To Feature Class) (conversion)
        parks_pointtojoin = arcpy.conversion.FeatureClassToFeatureClass(in_features=pois_area_, out_path=_area_others_gdb, out_name="parks_pointtojoin", where_clause="(code >= 2204 And code <= 2206)", field_mapping="code \"code\" true true false 4 Short 0 4,First,#,{pois_raw},code,-1,-1;fclass \"fclass\" true true false 28 Text 0 0,First,#,{pois_raw},fclass,0,28")[0]
        # Process: Merge parks (Merge) (management)
        arcpy.management.Merge(inputs=[greenspace_pointtojoin2, parks_pointtojoin], output=_area_parks.__str__().format(**globals()), field_mappings="code \"code\" true true false 0 Short 0 0,First,#,greenspace_pointtojoin2,code,-1,-1,parks_pointtojoin,code,-1,-1;fclass \"fclass\" true true false 255 Text 0 0,First,#,greenspace_pointtojoin2,fclass,0,255,parks_pointtojoin,fclass,0,28")
        # Directly use TableToTable conversion to overwrite the original table
        arcpy.conversion.TableToTable(_area_parks.__str__().format(**globals()), _area_others_gdb, os.path.basename(_area_parks.__str__().format(**globals())))
        
        #population
        # Process: Extract by Mask (Extract by Mask) (sa)
        _area_population_raw = fr"{folder}\{area}\{area}_others.gdb\{area}_population_raw"
        Extract_by_Mask = _area_population_raw
        _area_population_raw = arcpy.sa.ExtractByMask(population_raw.__str__().format(**globals()), _area_Buffer, "INSIDE", "501584.2233 153855.1252 563984.662575463 202984.882819595 PROJCS[\"British_National_Grid\",GEOGCS[\"GCS_OSGB_1936\",DATUM[\"D_OSGB_1936\",SPHEROID[\"Airy_1830\",6377563.396,299.3249646]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"False_Easting\",400000.0],PARAMETER[\"False_Northing\",-100000.0],PARAMETER[\"Central_Meridian\",-2.0],PARAMETER[\"Scale_Factor\",0.9996012717],PARAMETER[\"Latitude_Of_Origin\",49.0],UNIT[\"Meter\",1.0]]")
        _area_population_raw.save(Extract_by_Mask)
        # Process: Raster to Point (Raster to Point) (conversion)
        population_point = fr"{folder}\{area}\{area}_others.gdb\population_point"
        with arcpy.EnvManager(outputMFlag="Disabled", outputZFlag="Disabled"):
            arcpy.conversion.RasterToPoint(in_raster=_area_population_raw, out_point_features=population_point)
        # Process: Select more than 0 (Select) (analysis)
        population_point_Select = fr"{folder}\{area}\{area}_others.gdb\population_point_Select"
        arcpy.analysis.Select(in_features=population_point, out_feature_class=population_point_Select, where_clause="grid_code > 0")
        # Process: Intersect (Intersect) (analysis)
        arcpy.analysis.Intersect(in_features=[[_area_, ""], [population_point_Select, ""]], out_feature_class=_area_population.__str__().format(**globals()), join_attributes="ONLY_FID")
        # Define the desired output feature class
        output_fc = _area_population.__str__().format(**globals())
        arcpy.management.DeleteField(output_fc, [f.name for f in arcpy.ListFields(output_fc) if f.name not in ['OBJECTID', 'Shape', 'FID']])
        
        # roads
        # Process: Intersect roads (Intersect) (analysis)
        #arcpy.analysis.Intersect(in_features=[[_area_Buffer, ""], [roads_raw, ""]], out_feature_class=_area_roads.__str__().format(**globals()), join_attributes="NO_FID")
        
        # return
        # return analysis_area, _area_health, _area_leisure, _area_sports, _area_catering, _area_shopping, _area_supermarket, _area_financial, _area_education, _area_facilities, _area_worship, _area_transport, _area_parks, _area_roads, _area_population
        def snap_and_remove_duplicates(input_fc, snap_environment, search_distance="100 Meters"):
            # Snap points to the given snap environment
            arcpy.Snap_edit(input_fc, [[snap_environment, "VERTEX", search_distance]])
            # Use Delete Identical to remove duplicates based on their SHAPE (location)
            arcpy.DeleteIdentical_management(input_fc, ["SHAPE"])
            
        def process_geodatabase(gdb_path):
            # Find the 'others' geodatabase in the same directory as the input geodatabase
            directory = os.path.dirname(gdb_path)
            other_gdb = [os.path.join(directory, gdb) for gdb in os.listdir(directory) if 'others' in gdb and gdb.endswith('.gdb')][0]
            # Path to the 'population_point' feature class in the 'others' geodatabase
            snap_environment = os.path.join(other_gdb, "population_point")
            # Set the workspace to the input geodatabase
            arcpy.env.workspace = gdb_path
            # List all the feature classes in the geodatabase
            fcs = arcpy.ListFeatureClasses(feature_type='Point')
            # Process each point feature class
            for fc in fcs:
                snap_and_remove_duplicates(fc, snap_environment)
        process_geodatabase(_area_pois_gdb)
        
    #analysis_area, _area_health, _area_leisure, _area_sports, _area_catering, _area_shopping, _area_supermarket, _area_financial, _area_education, _area_facilities, _area_worship, _area_transport, _area_parks, _area_roads, _area_population = dataselect(folder, area, area_type, area_raw, population_raw, pois_raw, roads_raw, landuse_raw)
    dataselect(folder, area, area_type, area_raw, population_raw, pois_raw, roads_raw, landuse_raw)
    #return analysis_area, _area_health, _area_leisure, _area_sports, _area_catering, _area_shopping, _area_supermarket, _area_financial, _area_education, _area_facilities, _area_worship, _area_transport, _area_parks, _area_roads, _area_population
    
    
### run
with arcpy.EnvManager(outputCoordinateSystem="PROJCS[\"British_National_Grid\",GEOGCS[\"GCS_OSGB_1936\",DATUM[\"D_OSGB_1936\",SPHEROID[\"Airy_1830\",6377563.396,299.3249646]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"False_Easting\",400000.0],PARAMETER[\"False_Northing\",-100000.0],PARAMETER[\"Central_Meridian\",-2.0],PARAMETER[\"Scale_Factor\",0.9996012717],PARAMETER[\"Latitude_Of_Origin\",49.0],UNIT[\"Meter\",1.0]]", scratchWorkspace=folder, workspace=folder):
    datapreparation(folder, area, area_type)
    arcpy.SetParameter(3, True)
    arcpy.AddMessage("Success!")

### demandanalysis.py

***Parameters***

|Name|Data Type|Type|Direction|
|---|---|---|---|
|folder|Folder|Required|Input|
|area|String|Required|Input|
|success|Boolean|Derived|Output|

In [ ]:
### input parameter
folder=arcpy.GetParameterAsText(0)
area=arcpy.GetParameterAsText(1)


### import
import arcpy
from arcpy.sa import *
from sys import argv
import os


### environment
arcpy.env.overwriteOutput = True
arcpy.env.workspace = folder

        
### define functions
def demandcalculator(area, folder):
    _area_ = fr"{folder}\{area}"                
    # Process: Create File Geodatabase analysis (Create File Geodatabase) (management)
    _area_results_gdb = arcpy.management.CreateFileGDB(out_folder_path=_area_, out_name=f"{area}_results")[0]
    
    def FeatureClassGenerator(workspace, wild_card, feature_type, recursive) :
        with arcpy.EnvManager(workspace = workspace):
            dataset_list = [""]
            if recursive:
                datasets = arcpy.ListDatasets()
                dataset_list.extend(datasets)
            for dataset in dataset_list:
                featureclasses = arcpy.ListFeatureClasses(wild_card, feature_type, dataset)
                for fc in featureclasses:
                    yield os.path.join(workspace, dataset, fc), fc
                    
    def get_latest_feature_dataset_in_gdb(gdb_path):
            arcpy.env.workspace = gdb_path
            feature_datasets = arcpy.ListDatasets()
            if not feature_datasets:
                return None
            return feature_datasets[-1]
     
    def demandanalysis(walkdistance, area, folder, category):
        arcpy.ImportToolbox(r"C:\Users\xianl\Documents\ArcGIS\Projects\CASA0010\CASA0010.atbx")
        population = fr"{folder}\{area}\{area}_population.gdb\{area}_population"
        _area_pois_gdb = fr"{folder}\{area}\{area}_pois.gdb"
        Network_Data_Source = "https://www.arcgis.com/"
        IncidentID = "IncidentID"
        
        # Process: Create File Geodatabase analysis (Create File Geodatabase) (management)
        _area_analysis_gdb = arcpy.management.CreateFileGDB(out_folder_path=_area_, out_name=f"{area}_analysis")[0]
        _area_routes_gdb = arcpy.management.CreateFileGDB(out_folder_path=_area_, out_name=f"{area}_routes")[0]
        _area_others_gdb = arcpy.management.CreateFileGDB(out_folder_path=_area_, out_name=f"{area}_others")[0]
        _area_working_gdb = arcpy.management.CreateFileGDB(out_folder_path=_area_, out_name=f"{area}_working")[0]
        
        # Process: Project population (Project) (management)
        _area_population_Project = fr"{folder}\{area}\{area}_others.gdb\{area}_population_Project"
        arcpy.management.Project(in_dataset=population, out_dataset=_area_population_Project, out_coor_system="GEOGCS[\"GCS_WGS_1984\",DATUM[\"D_WGS_1984\",SPHEROID[\"WGS_1984\",6378137.0,298.257223563]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]]", in_coor_system="PROJCS[\"British_National_Grid\",GEOGCS[\"GCS_OSGB_1936\",DATUM[\"D_OSGB_1936\",SPHEROID[\"Airy_1830\",6377563.396,299.3249646]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"False_Easting\",400000.0],PARAMETER[\"False_Northing\",-100000.0],PARAMETER[\"Central_Meridian\",-2.0],PARAMETER[\"Scale_Factor\",0.9996012717],PARAMETER[\"Latitude_Of_Origin\",49.0],UNIT[\"Meter\",1.0]]")
        # Process: Buffer population (Buffer) (analysis)
        population_Buffer = fr"{folder}\{area}\{area}_others.gdb\{area}_population_Buffer"
        arcpy.analysis.Buffer(in_features=_area_population_Project, out_feature_class=population_Buffer, buffer_distance_or_field=f"{walkdistance} Kilometers", dissolve_option="ALL")
        
        for FeatureClass, pois in FeatureClassGenerator(_area_pois_gdb, "", "POINT", "NOT_RECURSIVE"):
            # Process: Table To Table (Table To Table) (conversion)
            _pois_demand = arcpy.conversion.TableToTable(in_rows=population, out_path=_area_others_gdb, out_name=f"{pois}_demand", field_mapping="ID \"ID\" true true false 255 Long 0 0,First,#," + population + ",OBJECTID,-1,-1")
            arcpy.env.workspace = fr"{folder}\{area}\{area}_working.gdb"
            # Process: Make Closest Facility Analysis Layer (Make Closest Facility Analysis Layer) (na)
            Closest_Facility_n_ = arcpy.na.MakeClosestFacilityAnalysisLayer(network_data_source=Network_Data_Source, layer_name=f"{pois}_{category}_closestfacility", travel_mode="Walking Distance", travel_direction="TO_FACILITIES", cutoff=walkdistance.__str__().format(**globals()), time_of_day="2023/9/1", line_shape="NO_LINES", ignore_invalid_locations="HALT")[0]
            # Process: Add population (Add Locations) (na)
            with_population = arcpy.na.AddLocations(in_network_analysis_layer=Closest_Facility_n_, sub_layer="Incidents", in_table=_area_population_Project, sort_field="OBJECTID", append="CLEAR")[0]
            # Process: Project facilities (Project) (management)
            _pois_Project = fr"{folder}\{area}\{area}_others.gdb\{pois}_Project"
            arcpy.management.Project(in_dataset=FeatureClass, out_dataset=_pois_Project, out_coor_system="GEOGCS[\"GCS_WGS_1984\",DATUM[\"D_WGS_1984\",SPHEROID[\"WGS_1984\",6378137.0,298.257223563]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]]")
            # Process: Intersect (Intersect) (analysis)
            facilities_inwalkdistance = fr"{folder}\{area}\{area}_others.gdb\{pois}_inwalkdistance"
            arcpy.analysis.Intersect(in_features=[[population_Buffer, ""], [_pois_Project, ""]], out_feature_class=facilities_inwalkdistance)
            # Process: Add facilities (Add Locations) (na)
            with_facilities = arcpy.na.AddLocations(in_network_analysis_layer=with_population, sub_layer="Facilities", in_table=facilities_inwalkdistance, sort_field="OBJECTID", append="CLEAR")[0]
            # Check if facilities_inwalkdistance is empty
            count = int(arcpy.GetCount_management(facilities_inwalkdistance).getOutput(0))
            if count == 0:
                # Create an empty feature class with specified fields
                arcpy.CreateFeatureclass_management(out_path=fr"{folder}\{area}\{area}_routes.gdb", out_name=f"{pois}_routes", geometry_type="POLYLINE")
                arcpy.AddField_management(in_table=fr"{folder}\{area}\{area}_routes.gdb\{pois}_routes", field_name="IncidentID", field_type="LONG")
                arcpy.AddField_management(in_table=fr"{folder}\{area}\{area}_routes.gdb\{pois}_routes", field_name="Total_WalkTime", field_type="DOUBLE")
            else:
                # Process: Solve (Solve) (na)
                Closest_Facility_result = arcpy.na.Solve(in_network_analysis_layer=with_facilities, ignore_invalids="SKIP", terminate_on_solve_error="TERMINATE", overrides=Network_Data_Source)
                def find_and_export_route_features(gdb_path, feature_dataset_name, output_folder):
                    arcpy.env.workspace = os.path.join(gdb_path, feature_dataset_name)
                    feature_classes = arcpy.ListFeatureClasses()
                    route_features = [fc for fc in feature_classes if 'Route' in fc]
                    for route_feature in route_features:
                        new_name = fr"{pois}_routes"
                        output_path = os.path.join(output_folder, new_name)
                        arcpy.CopyFeatures_management(route_feature, output_path) 
                latest_dataset = get_latest_feature_dataset_in_gdb(fr"{folder}\{area}\{area}_working.gdb")
                find_and_export_route_features(fr"{folder}\{area}\{area}_working.gdb", latest_dataset, fr"{folder}\{area}\{area}_routes.gdb")
                arcpy.Delete_management(os.path.join(fr"{folder}\{area}\{area}_working.gdb", latest_dataset))
            Routes = fr"{folder}\{area}\{area}_routes.gdb\{pois}_routes"
            # Process: export routes (Table To Table) (conversion)
            _pois_routes = arcpy.conversion.TableToTable(
                in_rows=Routes,
                out_path=_area_others_gdb,
                out_name=f"{pois}_routes",
                field_mapping="IncidentID \"IncidentID\" true true true 4 Long 0 0,First,#,ModelBuilder\\Routes:Routes,IncidentID,-1,-1;Total_WalkTime \"Total_WalkTime\" true true true 8 Double 0 0,First,#,ModelBuilder\\Routes:Routes,Total_WalkTime,-1,-1"
            )[0]
            # Process: Join Field (Join Field) (management)
            _pois_demand_2_ = arcpy.management.JoinField(
                in_data=_pois_demand,
                in_field="ID",
                join_table=_pois_routes,
                join_field="IncidentID",
                fields=["Total_WalkTime"]
            )[0]
            _pois_category_demandresult = arcpy.conversion.TableToTable(
                in_rows=_pois_demand_2_,
                out_path=_area_analysis_gdb,
                out_name=f"{pois}_{category}_demandresult",
            )[0]
            arcpy.AddField_management(in_table=_pois_category_demandresult, field_name=f"{pois}_demand", field_type="FLOAT")
            result_calculate = arcpy.management.CalculateField(
                in_table=_pois_category_demandresult,
                field=f"{pois}_demand",
                expression="0.0 if !Total_WalkTime! == None else math.exp(-0.073 * !Total_WalkTime!)",
                expression_type="PYTHON3",
                code_block="import math"
            )[0]
            # Process: Delete Field (Delete Field) (management)
            Update_result = arcpy.management.DeleteField(in_table=result_calculate, drop_field=[IncidentID])[0]
       
        # define demandcalculate
        def demandcalculate(output_table_path):
            input_gdb = fr"{folder}\{area}\{area}_analysis.gdb"
            arcpy.env.workspace = input_gdb
            tables = arcpy.ListTables()
            arcpy.Copy_management(tables[0], output_table_path)
            arcpy.DeleteField_management(output_table_path, ["ID", "Total_WalkTime"])

            for table in tables[1:]:
                fields = arcpy.ListFields(table)
                second_column = fields[3].name
                arcpy.JoinField_management(output_table_path, "OBJECTID", table, "OBJECTID", [second_column])
            fields = arcpy.ListFields(output_table_path)
            sum_fields = [f.name for f in fields if f.name != 'OBJECTID']
            
            # Add new field for demandresult and calculate its value
            arcpy.AddField_management(output_table_path, "demandresult", "FLOAT")
            with arcpy.da.UpdateCursor(output_table_path, sum_fields + ["demandresult"]) as cursor:
                for row in cursor:
                    total = sum(row[:-1])
                    average = total / 12.0
                    row[-1] = average
                    cursor.updateRow(row)
            
        # Process: demandcalculate (demandcalculate)     
        demandcalculate(output_table_path=fr"{folder}\{area}\{area}_results.gdb\{area}_{category}_demandresult")
        arcpy.JoinField_management(fr"{folder}\{area}\{area}_population.gdb\{area}_population", "OBJECTID", fr"{folder}\{area}\{area}_results.gdb\{area}_{category}_demandresult", "OBJECTID")

    with arcpy.da.SearchCursor(f"{folder}\\input\\walkdistance.csv", ['category', 'distance15']) as cursor:
        for row in cursor:
            category = row[0]
            walkdistance = row[1]
            if category == 'standard':  # only calculate standard category
                demandanalysis(walkdistance, area, folder, category)
                break  # only calculate standard category


### run
with arcpy.EnvManager(outputCoordinateSystem="PROJCS[\"British_National_Grid\",GEOGCS[\"GCS_OSGB_1936\",DATUM[\"D_OSGB_1936\",SPHEROID[\"Airy_1830\",6377563.396,299.3249646]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"False_Easting\",400000.0],PARAMETER[\"False_Northing\",-100000.0],PARAMETER[\"Central_Meridian\",-2.0],PARAMETER[\"Scale_Factor\",0.9996012717],PARAMETER[\"Latitude_Of_Origin\",49.0],UNIT[\"Meter\",1.0]]", scratchWorkspace=folder, workspace=folder):
    demandcalculator(area, folder)
    arcpy.SetParameter(2, True)
    arcpy.AddMessage("Success!")

### supplyanalysis.py

***Parameters***

|Name|Data Type|Type|Direction|
|---|---|---|---|
|folder|Folder|Required|Input|
|area|String|Required|Input|
|area_type|String|Required|Input|
|success|Boolean|Derived|Output|

In [ ]:
### import
import arcpy
import os
from sys import argv


### input parameter
folder=arcpy.GetParameterAsText(0)
area=arcpy.GetParameterAsText(1)
area_type=arcpy.GetParameterAsText(2)


### environment
arcpy.env.workspace = folder
arcpy.env.overwriteOutput = True


### define functions
def split_by_count(input_fc, count=999):
    # Get a list of OBJECTIDs from the input feature class
    objectids = [row[0] for row in arcpy.da.SearchCursor(input_fc, 'OBJECTID')]
    total_count = len(objectids)
    # If the total count is less than or equal to the specified count, exit the function
    if total_count <= count:
        return
    # Create a feature layer from the input feature class
    arcpy.management.MakeFeatureLayer(input_fc, "temp_layer")
    # Retrieve the directory and the name of the input feature class
    folder_path, fc_name = os.path.split(input_fc)
    fc_name_noext = os.path.splitext(fc_name)[0]  # Get the name without the extension
    start_index = 0
    chunk_num = 1
    while start_index < total_count:
        end_index = start_index + count
        if end_index > total_count:
            end_index = total_count
        current_ids = objectids[start_index:end_index]
        sql_query = "OBJECTID IN ({})".format(','.join(map(str, current_ids)))
        # Select the subset of features
        arcpy.management.SelectLayerByAttribute("temp_layer", "NEW_SELECTION", sql_query)
        # Name for the output feature class
        output_fc = os.path.join(folder_path, f"{fc_name_noext}_{chunk_num}")
        # Export the selected features to a new feature class
        arcpy.management.CopyFeatures("temp_layer", output_fc)
        start_index = end_index
        chunk_num += 1
    # Remove the original feature class and the temporary feature layer
    arcpy.management.Delete(input_fc)
    arcpy.management.Delete("temp_layer")

def process_geodatabase(gdb_path):
    # Set the workspace environment
    arcpy.env.workspace = gdb_path
    # List all the feature classes in the geodatabase
    fcs = arcpy.ListFeatureClasses()
    # Process each feature class
    for fc in fcs:
        split_by_count(fc)

def FeatureClassGenerator(workspace, wild_card, feature_type, recursive) :
    with arcpy.EnvManager(workspace = workspace):
        dataset_list = [""]
        if recursive:
            datasets = arcpy.ListDatasets()
            dataset_list.extend(datasets)
            for dataset in dataset_list:
                featureclasses = arcpy.ListFeatureClasses(wild_card, feature_type, dataset)
                for fc in featureclasses:
                    yield os.path.join(workspace, dataset, fc), fc

def get_latest_feature_dataset_in_gdb(gdb_path):
    arcpy.env.workspace = gdb_path
    feature_datasets = arcpy.ListDatasets()
    if not feature_datasets:
        return None
    return feature_datasets[-1]
                    
def supplyanalysis(_area_, _folder_):
    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = True
    Network_Data_Source = "https://www.arcgis.com/"
    _area_pois_gdb = fr"{folder}\{area}\{area}_pois.gdb"
    _area_ = fr"{folder}\{area}"
    _area_sa_gdb = arcpy.management.CreateFileGDB(out_folder_path=_area_, out_name=f"{area}_sa")[0]
    _area_working_gdb = arcpy.management.CreateFileGDB(out_folder_path=_area_, out_name=f"{area}_working")[0]
    _area_others_gdb = arcpy.management.CreateFileGDB(out_folder_path=_area_, out_name=f"{area}_others")[0]
    for pois_point, pois in FeatureClassGenerator(_area_pois_gdb, "", "", "NOT_RECURSIVE"):
        # Process: Make Service Area Analysis Layer (Make Service Area Analysis Layer) (na)
        arcpy.env.workspace = fr"{folder}\{area}\{area}_working.gdb"
        Service_Area_layer = arcpy.na.MakeServiceAreaAnalysisLayer(network_data_source=Network_Data_Source, travel_mode="Walking Time", cutoffs=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], time_of_day="2023/9/1", polygon_detail="STANDARD", geometry_at_overlaps="OVERLAP", polygon_trim_distance="50 Meters")[0]     
        # Process: Add Facilities (Add Locations) (na)
        Service_Area_layer_with_pois = arcpy.na.AddLocations(in_network_analysis_layer=Service_Area_layer, sub_layer="Facilities", in_table=pois_point)[0]
        # Check if pois_point is empty
        feature_count = int(arcpy.GetCount_management(pois_point).getOutput(0))
        if feature_count == 0:
            # Create an empty polygon feature class named sa_{pois} with two float fields
            output_fc_path = fr"{folder}\{area}\{area}_sa.gdb\sa_{pois}"
            arcpy.CreateFeatureclass_management(
                out_path=fr"{folder}\{area}\{area}_sa.gdb",
                out_name=f"sa_{pois}",
                geometry_type="POLYGON"
            )
            arcpy.AddField_management(output_fc_path, "FromBreak", "FLOAT")
            arcpy.AddField_management(output_fc_path, "ToBreak", "FLOAT")
        else:
            # Process: Solve (Solve) (na)
            Service_Area_result, Solve_Succeeded = arcpy.na.Solve(in_network_analysis_layer=Service_Area_layer_with_pois)
            # Process: Select_Data (Select Data)      
            def find_and_export_sa_features(gdb_path, feature_dataset_name, output_folder):
                arcpy.env.workspace = os.path.join(gdb_path, feature_dataset_name)
                feature_classes = arcpy.ListFeatureClasses()
                sa_features = [fc for fc in feature_classes if 'SAPolygons' in fc]
                for sa_feature in sa_features:
                    new_name = fr"sa_{pois}"
                    output_path = os.path.join(output_folder, new_name)
                    arcpy.CopyFeatures_management(sa_feature, output_path)
            latest_dataset = get_latest_feature_dataset_in_gdb(fr"{folder}\{area}\{area}_working.gdb")
            find_and_export_sa_features(fr"{folder}\{area}\{area}_working.gdb", latest_dataset, fr"{folder}\{area}\{area}_sa.gdb")
            arcpy.Delete_management(os.path.join(fr"{folder}\{area}\{area}_working.gdb", latest_dataset))

# auto merge
def get_common_name(fcs):
    if len(fcs) >= 2:
        return os.path.commonprefix(fcs).rstrip('_')
    else:
        return None

def group_feature_classes_by_field(gdb):
    arcpy.env.workspace = gdb
    grouped_fcs = {}
    # List all feature classes in the geodatabase
    fcs = arcpy.ListFeatureClasses()
    # Iterate over feature classes and group by the field between underscores
    for fc in fcs:
        parts = fc.split('_')
        if len(parts) > 2:
            key_field = parts[2]
            if key_field not in grouped_fcs:
                grouped_fcs[key_field] = []
            grouped_fcs[key_field].append(fc)
    return grouped_fcs

def merge_and_rename_feature_classes_by_group(gdb):
    grouped_fcs = group_feature_classes_by_field(gdb)
    for key, fcs in grouped_fcs.items():
        common_name = get_common_name(fcs)
        if common_name:
            # Merge the feature classes
            temp_output_fc = f"{common_name}_merged"
            arcpy.Merge_management(fcs, temp_output_fc)
            # Rename the merged feature class by removing everything after the last underscore
            final_name = "_".join(temp_output_fc.split('_')[:-1])
            arcpy.Rename_management(temp_output_fc, final_name)
            # Delete the source feature classes
            for fc in fcs:
                arcpy.Delete_management(fc)

def supplycalculation(folder, area, area_type):
    def FeatureClassGenerator(workspace, wild_card, feature_type, recursive) :
            with arcpy.EnvManager(workspace = workspace):
                dataset_list = [""]
                if recursive:
                    datasets = arcpy.ListDatasets()
                    dataset_list.extend(datasets)
                for dataset in dataset_list:
                    featureclasses = arcpy.ListFeatureClasses(wild_card, feature_type, dataset)
                    for fc in featureclasses:
                        yield os.path.join(workspace, dataset, fc), fc
    # Process: Dissolve (Dissolve) (management)
    _area_area_type_dissolve = fr"{folder}\{area}\{area}_others.gdb\{area}_{area_type}_dissolve"
    arcpy.management.Dissolve(in_features=fr"{folder}\{area}\{area}_boundaries.gdb\{area}_{area_type}", out_feature_class=_area_area_type_dissolve)
            
    # supplycalculate
    def supplycalculate(folder, area, area_type):
        # To allow overwriting outputs change overwriteOutput option to True.
        arcpy.env.overwriteOutput = True
        _area_population = fr"{folder}\{area}\{area}_population.gdb\{area}_population"
        _area_sa_gdb = fr"{folder}\{area}\{area}_sa.gdb"
        _area_area_type_dissolve = fr"{folder}\{area}\{area}_others.gdb\{area}_{area_type}_dissolve"
        for pois_supply, pois in FeatureClassGenerator(_area_sa_gdb, "", "", "NOT_RECURSIVE"):
            # Process: Calculate time (Calculate Field) (management)
            pois_supply_1_ = arcpy.management.CalculateField(in_table=pois_supply, field="time", expression="($feature.FromBreak + $feature.ToBreak) / 2", expression_type="ARCADE", field_type="DOUBLE")[0]
            # Process: Calculate supply (Calculate Field) (management)
            pois_supply_2_ = arcpy.management.CalculateField(in_table=pois_supply_1_, field=f"{pois}_supply", expression="0 if !time! is None else float(math.exp(-0.073 * !time!))", code_block="import math", field_type="DOUBLE")[0]
            # Process: Spatial Join (Spatial Join) (analysis)
            _area_population_2_ = fr"{folder}\{area}\{area}_analysis.gdb\{pois}_population"
            field_mapping = (
                f"\"{pois}_supply\" "
                f"\"{pois}_supply\" "
                "true true false "
                "4 Float 0 0 ,"
                "Sum,"
                "#,"
                fr"\"{folder}\\{area}\\{area}_sa.gdb\\sa_{pois}\","
                f"\"{pois[:2].upper()}{pois[2:]}_supply\","
                "-1,-1"
            )
            arcpy.analysis.SpatialJoin(target_features=_area_population, join_features=pois_supply_2_, out_feature_class=_area_population_2_, join_operation="JOIN_ONE_TO_ONE", field_mapping=field_mapping)
    supplycalculate(folder, area, area_type)    
    
    def process_fields_and_data(folder, area):
        area_analysis_gdb = fr"{folder}\{area}\{area}_analysis.gdb"
        population_gdb = fr"{folder}\{area}\{area}_population.gdb\{area}_population"
        
        for pois_pop, _ in FeatureClassGenerator(area_analysis_gdb, "*_population", "", "NOT_RECURSIVE"):
            supply_fields = [f.name for f in arcpy.ListFields(pois_pop) if 'sa' in f.name.lower()]
            for field in supply_fields:
                old_field_name = field
                new_field_name = field.split("_", 1)[-1]
                arcpy.JoinField_management(in_data=population_gdb, in_field="OBJECTID", join_table=pois_pop, join_field="OBJECTID", fields=[field])            
                arcpy.AlterField_management(population_gdb, old_field_name, new_field_name, new_field_name)
                with arcpy.da.UpdateCursor(population_gdb, new_field_name) as cursor:
                    for row in cursor:
                        if row[0] is None:
                            row[0] = 0
                        cursor.updateRow(row)
    
        supply_fields = [f.name for f in arcpy.ListFields(population_gdb) if 'supply' in f.name.lower()]
        if "supplyresult" not in [f.name for f in arcpy.ListFields(population_gdb)]:
            arcpy.AddField_management(population_gdb, "supplyresult", "DOUBLE")
        
        calculate_expression = "({})/{}".format(" + ".join(["!" + f + "!" for f in supply_fields]), len(supply_fields))
        arcpy.CalculateField_management(population_gdb, "supplyresult", calculate_expression, "PYTHON3")
        
        all_fields = [f.name for f in arcpy.ListFields(population_gdb)]
    
        for field in all_fields:
            if "_" in field:
                new_field_name = field.split("_", 1)[-1]
                arcpy.AlterField_management(population_gdb, field, new_field_name, new_field_name)
    
        output_gdb = fr"{folder}\{area}\{area}_finalresult.gdb"
        if not arcpy.Exists(output_gdb):
            arcpy.CreateFileGDB_management(fr"{folder}\{area}", f"{area}_finalresult.gdb")
        
        arcpy.FeatureClassToFeatureClass_conversion(population_gdb, output_gdb, f"{area}_result")
    process_fields_and_data(folder, area)
      

### run
with arcpy.EnvManager(outputCoordinateSystem="PROJCS[\"British_National_Grid\",GEOGCS[\"GCS_OSGB_1936\",DATUM[\"D_OSGB_1936\",SPHEROID[\"Airy_1830\",6377563.396,299.3249646]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"False_Easting\",400000.0],PARAMETER[\"False_Northing\",-100000.0],PARAMETER[\"Central_Meridian\",-2.0],PARAMETER[\"Scale_Factor\",0.9996012717],PARAMETER[\"Latitude_Of_Origin\",49.0],UNIT[\"Meter\",1.0]]", scratchWorkspace=folder, workspace=folder):
    gdb_input = fr"{folder}\{area}\{area}_pois.gdb"
    process_geodatabase(gdb_input)
    supplyanalysis(area, folder)
    merge_and_rename_feature_classes_by_group(fr"{folder}\{area}\{area}_sa.gdb")
    supplycalculation(folder, area, area_type)
    arcpy.SetParameter(3, True)
    arcpy.AddMessage("Success!")

### resultcalculation.py

***Parameters***

|Name|Data Type|Type|Direction|
|---|---|---|---|
|folder|Folder|Required|Input|


In [ ]:
### import
import arcpy
from arcpy.sa import *
from sys import argv
import arcpy
from sklearn.decomposition import PCA
import numpy as np
from scipy.stats import zscore
import os


### input parameter
folder = arcpy.GetParameterAsText(0)


### environment
arcpy.env.workspace = folder
arcpy.env.overwriteOutput = True


### functions
  
# merge result
def merge_featureclasses_from_gdbs(folder):
    all_featureclasses = []
    for area in os.listdir(folder):
        gdb_path = fr"{folder}\{area}\{area}_finalresult.gdb"
        if os.path.exists(gdb_path):
            arcpy.env.workspace = gdb_path
            featureclasses = arcpy.ListFeatureClasses()
            for fc in featureclasses:
                all_featureclasses.append(os.path.join(gdb_path, fc))
    if all_featureclasses:
        arcpy.CreateFileGDB_management(folder, "output.gdb")
        output_gdb = os.path.join(folder, "output.gdb")
        output_fc_path = os.path.join(output_gdb, "convenience")
        arcpy.Merge_management(all_featureclasses, output_fc_path)

# delete gdbs
def delete_specific_gdbs(folder):
    for area in os.listdir(folder):
        area_folder = os.path.join(folder, area)
        if os.path.isdir(area_folder):
            for gdb_name in os.listdir(area_folder):
                if any(keyword in gdb_name.lower() for keyword in ['others', 'analysis', 'working', 'results']):
                    gdb_path = os.path.join(area_folder, gdb_name)
                    if arcpy.Exists(gdb_path) and arcpy.Describe(gdb_path).workspaceType == "FileGDB":
                        arcpy.Delete_management(gdb_path)

merge_featureclasses_from_gdbs(folder)
delete_specific_gdbs(folder)

fc = fr"{folder}\output.gdb\convenience"
all_fields = [f.name for f in arcpy.ListFields(fc)]
demand_fields = [f for f in all_fields if 'demand' in f.lower()]
supply_fields = [f for f in all_fields if '_supply' in f.lower()]
supplyresult_fields = [f for f in all_fields if 'supplyresult' in f.lower()]

# Gather values from all the _supply fields
all_supply_values = []

for supply_field in supply_fields:
    values = [row[0] for row in arcpy.da.SearchCursor(fc, supply_field)]
    all_supply_values.extend(values)

# Calculate the overall z-scores
z_scores = zscore(all_supply_values)

# Determine the range for non-outlier values based on the z-scores
threshold = 3
valid_indices = np.where(np.abs(z_scores) <= threshold)[0]
valid_values = np.array(all_supply_values)[valid_indices]
max_non_outlier = valid_values.max()
min_non_outlier = valid_values.min()

# Apply the non-outlier range to each _supply field
for supply_field in supply_fields:
    with arcpy.da.UpdateCursor(fc, supply_field) as cursor:
        for row in cursor:
            if row[0] > max_non_outlier:
                row[0] = max_non_outlier
            elif row[0] < min_non_outlier:
                row[0] = min_non_outlier
            cursor.updateRow(row)

# Before the transformation, set _supply to 0 when corresponding _demand is 0
for demand_field in demand_fields:
    supply_field = demand_field.replace("_demand", "_supply")
    with arcpy.da.UpdateCursor(fc, [demand_field, supply_field]) as cursor:
        for row in cursor:
            if row[0] == 0:
                row[1] = 0
            cursor.updateRow(row)

# Use the max_non_outlier as the max_value_all_fields for logarithmic transformation
max_value_all_fields = max_non_outlier

# Next, transform the _supply fields using logarithms
for supply_field in supply_fields:
    values = [row[0] for row in arcpy.da.SearchCursor(fc, supply_field)]
    # Add 1 to each value, then take its logarithm
    transformed_values = np.log(np.array(values) + 1) / np.log(max_value_all_fields + 1)

    # Update the field values
    with arcpy.da.UpdateCursor(fc, supply_field) as cursor:
        for i, row in enumerate(cursor):
            row[0] = transformed_values[i]
            cursor.updateRow(row)
            
# Update the 'supplyresult' field with the average of all _supply fields
with arcpy.da.UpdateCursor(fc, supply_fields + ['supplyresult']) as cursor:
    for row in cursor:
        # Calculate average, excluding None values
        avg_value = sum(val for val in row[:-1] if val is not None) / len([val for val in row[:-1] if val is not None])
        row[-1] = avg_value
        cursor.updateRow(row)
        
# Identify pairs of _demand and _supply fields
pair_fields = {}
for demand_field in demand_fields:
    prefix = demand_field.replace("_demand", "")
    corresponding_supply_field = prefix + "_supply"
    if corresponding_supply_field in supply_fields:
        pair_fields[demand_field] = corresponding_supply_field

# Create a new average field for each pair
new_avg_fields = []  # Store the newly created average field names for later use
for demand_field, supply_field in pair_fields.items():
    avg_field_name = f"{demand_field.replace('_demand', '')}"
    arcpy.AddField_management(fc, avg_field_name, "DOUBLE")
    new_avg_fields.append(avg_field_name)

    with arcpy.da.UpdateCursor(fc, [demand_field, supply_field, avg_field_name]) as cursor:
        for row in cursor:
            # Calculate the average, ensuring both fields have values
            if row[0] is not None and row[1] is not None:
                row[2] = (row[0] + row[1]) / 2
                cursor.updateRow(row)

# Create an overall average field from the newly generated average fields
arcpy.AddField_management(fc, "average", "DOUBLE")
with arcpy.da.UpdateCursor(fc, new_avg_fields + ["average"]) as cursor:
    for row in cursor:
        avg_values = [val for val in row[:-1] if val is not None]
        if avg_values:  # Prevent division by zero
            total_avg = sum(avg_values) / len(avg_values)
            row[-1] = total_avg
            cursor.updateRow(row)
            
def joinpopulation(folder): 
    arcpy.env.overwriteOutput = True
    # Check out any necessary licenses.
    arcpy.CheckOutExtension("spatial")
    convenience = fr"{folder}\output.gdb\convenience"
    population_tif = arcpy.Raster(fr"{folder}\input\population.tif")
    sedata_shp = fr"{folder}\input\meanprice.shp"
    # Process: Create File Geodatabase (Create File Geodatabase) (management)
    analysis_gdb = arcpy.management.CreateFileGDB(out_folder_path=folder.__str__().format(**globals()), out_name="analysis")[0]
    # Process: Extract by Mask (Extract by Mask) (sa)
    population = fr"{folder}\analysis.gdb\population"
    Extract_by_Mask = population
    with arcpy.EnvManager(scratchWorkspace=analysis_gdb):
        population = arcpy.sa.ExtractByMask(population_tif, sedata_shp)
    # Process: Extract Values to Points (Extract Values to Points) (sa)
    convenience_population = fr"{folder}\analysis.gdb\convenience_population"
    arcpy.sa.ExtractValuesToPoints(convenience, population, convenience_population, "NONE", "VALUE_ONLY")

def multiply_fields_by_population_column_and_add_pop(fc):
    all_field_names = [f.name for f in arcpy.ListFields(fc)]
    arcpy.AlterField_management(fc, 'RASTERVALU', "population", "population")
    all_field_names = [f.name for f in arcpy.ListFields(fc)]
    fields_without_special = [f for f in all_field_names if f not in ["OBJECTID", "Shape", "population"]]
    all_field_names = [f.name for f in arcpy.ListFields(fc)]
    with arcpy.da.UpdateCursor(fc, "population") as cursor:
        for row in cursor:
            if row[0] is None:
                row[0] = 0
            cursor.updateRow(row)
    for field in fields_without_special:
        new_field_name = field + "_pop"
        if new_field_name not in all_field_names:
            arcpy.AddField_management(fc, new_field_name, "DOUBLE")
            all_field_names.append(new_field_name)
    fields_to_update = fields_without_special + ["population"] + [f + "_pop" for f in fields_without_special]
    with arcpy.da.UpdateCursor(fc, fields_to_update) as cursor:
        for row in cursor:
            for i, field in enumerate(fields_without_special):
                row_value = float(row[i]) * float(row[len(fields_without_special)])
                row[i + len(fields_without_special) + 1] = row_value
            cursor.updateRow(row)

def spatial_join_and_sum_points_to_polygon(points_fc, polygon_fc):
    output_polygon = fr"{folder}\output.gdb\convenience_area"
    all_field_names = [f.name for f in arcpy.ListFields(points_fc)]
    fields_with_pop = [f for f in all_field_names if "pop" in f]
    field_mappings = arcpy.FieldMappings()
    for field in fields_with_pop:
        fm = arcpy.FieldMap()
        fm.addInputField(points_fc, field)
        f_out = fm.outputField
        f_out.name = field
        fm.outputField = f_out
        fm.mergeRule = 'Sum'
        field_mappings.addFieldMap(fm)
    arcpy.analysis.SpatialJoin(target_features=polygon_fc, 
                               join_features=points_fc, 
                               out_feature_class=output_polygon, 
                               join_type="KEEP_COMMON", 
                               field_mapping=field_mappings, 
                               match_option="INTERSECT")

def divide_fields_by_population_and_rename(output_polygon):
    all_field_names = [f.name for f in arcpy.ListFields(output_polygon)]
    fields_with_pop = [f for f in all_field_names if "_pop" in f]
    if 'population' in all_field_names:
        for field in fields_with_pop:
            new_field_name = field.replace("_pop", "")
            arcpy.AddField_management(output_polygon, new_field_name, "DOUBLE")
            with arcpy.da.UpdateCursor(output_polygon, [field, 'population', new_field_name]) as cursor:
                for row in cursor:
                    if row[1] != 0:
                        row[2] = row[0] / row[1]
                    else:
                        row[2] = None
                    cursor.updateRow(row)
            arcpy.DeleteField_management(output_polygon, field)
    fields_to_delete = ['Join_Count', 'TARGET_FID']
    for field in fields_to_delete:
        if field in all_field_names:
            arcpy.DeleteField_management(output_polygon, field)

def spatial_join_meanprice_to_convenience_area():
    target_features = fr"{folder}\output.gdb\convenience_area"
    join_features = fr"{folder}\input\meanprice.shp"
    output_features = fr"{folder}\output.gdb\convenience_area_update"
    field_mappings = arcpy.FieldMappings()
    field_mappings.addTable(target_features)
    for field in ["LSOA21CD", "meanprice"]:
        fm = arcpy.FieldMap()
        fm.addInputField(join_features, field)
        field_mappings.addFieldMap(fm)
    arcpy.analysis.SpatialJoin(target_features=target_features, 
                               join_features=join_features, 
                               out_feature_class=output_features, 
                               join_type="KEEP_ALL", 
                               field_mapping=field_mappings, 
                               match_option="ARE_IDENTICAL_TO")
    existing_fields = [field.name for field in arcpy.ListFields(output_features)]
    unwanted_fields = ['Join_Count', 'TARGET_FID']
    for field in unwanted_fields:
        if field in existing_fields:
            arcpy.DeleteField_management(output_features, field)
    arcpy.management.Delete(target_features)
    arcpy.management.Rename(output_features, target_features)


### run
with arcpy.EnvManager(outputCoordinateSystem="PROJCS[\"British_National_Grid\",GEOGCS[\"GCS_OSGB_1936\",DATUM[\"D_OSGB_1936\",SPHEROID[\"Airy_1830\",6377563.396,299.3249646]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"False_Easting\",400000.0],PARAMETER[\"False_Northing\",-100000.0],PARAMETER[\"Central_Meridian\",-2.0],PARAMETER[\"Scale_Factor\",0.9996012717],PARAMETER[\"Latitude_Of_Origin\",49.0],UNIT[\"Meter\",1.0]]", scratchWorkspace=folder, workspace=folder):
    polygon_fc = fr"{folder}\input\meanprice.shp"
    joinpopulation(folder)
    multiply_fields_by_population_column_and_add_pop(fr"{folder}\analysis.gdb\convenience_population")
    spatial_join_and_sum_points_to_polygon(fr"{folder}\analysis.gdb\convenience_population", polygon_fc)
    divide_fields_by_population_and_rename(fr"{folder}\output.gdb\convenience_area")
    spatial_join_meanprice_to_convenience_area()
    arcpy.SetParameter(1, True)
    arcpy.AddMessage("Success!")

### censusandcluster.py

***Parameters***

|Name|Data Type|Type|Direction|
|---|---|---|---|
|folder|Folder|Required|Input|
|success|Boolean|Derived|Output|

In [ ]:
### import
import pandas as pd
import numpy as np
import os
import arcpy


### input parameter
folder=arcpy.GetParameterAsText(0)


### environment
arcpy.env.overwriteOutput = True
arcpy.env.workspace = fr"{folder}\input\census"


### functions
avg_age_raw = pd.read_csv(fr"{folder}\input\census\RM200-Sex-By-Single-Year-Of-Age-(Detailed)-2021-lsoa-ONS.csv")
grouped = avg_age_raw.groupby(['Lower layer Super Output Areas Code', 'Age (91 categories) Code']).agg({'Observation': 'sum'}).reset_index()
grouped['Weighted Age'] = grouped['Age (91 categories) Code'] * grouped['Observation']
avg_age = grouped.groupby('Lower layer Super Output Areas Code').apply(lambda x: x['Weighted Age'].sum() / x['Observation'].sum()).reset_index()
avg_age.columns = ['LSOA_code', 'age']
print(avg_age)
avg_age.to_csv("age.csv", index=False)

percentage_non_uk_raw = pd.read_csv(fr"{folder}\input\census\TS004-Country-Of-Birth-2021-lsoa-ONS.csv")
non_uk_population = percentage_non_uk_raw[percentage_non_uk_raw['Country of birth (12 categories)'] != 'Europe: United Kingdom'].groupby('Lower Layer Super Output Areas Code')['Observation'].sum()
total_population = percentage_non_uk_raw.groupby('Lower Layer Super Output Areas Code')['Observation'].sum()
percentage_non_uk = (non_uk_population / total_population).reset_index()
percentage_non_uk.columns = ['LSOA_code', 'nonlocal']
print(percentage_non_uk)
percentage_non_uk.to_csv("nonlocal.csv", index=False)

percentage_less_than_5_years_raw = pd.read_csv(fr"{folder}\input\census\TS016-Length-Of-Residence-2021-lsoa-ONS.csv")
less_than_5_years = percentage_less_than_5_years_raw[percentage_less_than_5_years_raw['Length of residence in the UK (6 categories)'].isin(['2 years or more, but less than 5 years', 'Less than 2 years'])]
population_less_than_5_years = less_than_5_years.groupby('Lower layer Super Output Areas Code')['Observation'].sum()
total_population = percentage_less_than_5_years_raw.groupby('Lower layer Super Output Areas Code')['Observation'].sum()
percentage_less_than_5_years = (population_less_than_5_years / total_population).reset_index()
percentage_less_than_5_years.columns = ['LSOA_code', 'mobility']
print(percentage_less_than_5_years)
percentage_less_than_5_years.to_csv("mobility.csv", index=False)

average_household_size_raw = pd.read_csv(fr"{folder}\input\census\TS017-Household-Size-2021-lsoa-ONS.csv")
average_household_size_raw['Total People for Household Size'] = average_household_size_raw['Household size (9 categories) Code'] * average_household_size_raw['Observation']
total_people_per_area = average_household_size_raw.groupby('Lower layer Super Output Areas Code')['Total People for Household Size'].sum()
total_households_per_area = average_household_size_raw.groupby('Lower layer Super Output Areas Code')['Observation'].sum()
average_household_size = (total_people_per_area / total_households_per_area).reset_index()
average_household_size.columns = ['LSOA_code', 'household']
print(average_household_size)
average_household_size.to_csv("household.csv", index=False)

diversity_index_raw = pd.read_csv(fr"{folder}\input\census\TS021-Ethnic-Group-2021-lsoa-ONS.csv")
total_population_per_area = diversity_index_raw.groupby('Lower layer Super Output Areas Code')['Observation'].sum()
def shannon_diversity(group):
    proportions = group['Observation'] / total_population_per_area[group.name]
    return -np.sum(proportions * np.log(proportions))
diversity_index = diversity_index_raw.groupby('Lower layer Super Output Areas Code').apply(shannon_diversity).reset_index()
diversity_index.columns = ['LSOA_code', 'ethnic']
print(diversity_index)
diversity_index.to_csv("ethnic.csv", index=False)

faithful_proportion_raw = pd.read_csv(fr"{folder}\input\census\TS030-Religion-2021-lsoa-ONS.csv")
total_population_per_area = faithful_proportion_raw.groupby('Lower layer Super Output Areas Code')['Observation'].sum()
faithful_proportion_raw_faithful = faithful_proportion_raw[~faithful_proportion_raw['Religion (10 categories) Code'].isin([1])]
faithful_population_per_area = faithful_proportion_raw_faithful.groupby('Lower layer Super Output Areas Code')['Observation'].sum()
faithful_proportion = (faithful_population_per_area / total_population_per_area).reset_index()
faithful_proportion.columns = ['LSOA_code', 'religion']
print(faithful_proportion)
faithful_proportion.to_csv("religion.csv", index=False)

car_ownership_proportion_raw = pd.read_csv(fr"{folder}\input\census\TS045-Car-Or-Van-Availability-2021-lsoa-ONS.csv")
car_ownership_proportion_raw_cars = car_ownership_proportion_raw[~car_ownership_proportion_raw['Car or van availability (5 categories) Code'].isin([-8, 0])]
car_ownership_per_area = car_ownership_proportion_raw_cars.groupby('Lower layer Super Output Areas Code')['Observation'].sum()
total_households_per_area = car_ownership_proportion_raw.groupby('Lower layer Super Output Areas Code')['Observation'].sum()
car_ownership_proportion = (car_ownership_per_area / total_households_per_area).reset_index()
car_ownership_proportion.columns = ['LSOA_code', 'car']
print(car_ownership_proportion)
car_ownership_proportion.to_csv("car.csv", index=False)

average_rooms_per_area_raw = pd.read_csv(fr"{folder}\input\census\TS051-Number-Of-Rooms-2021-lsoa-ONS.csv")
average_rooms_per_area_raw['weighted_rooms'] = average_rooms_per_area_raw['Number of rooms (Valuation Office Agency) (9 categories) Code'] * average_rooms_per_area_raw['Observation']
total_houses_per_area = average_rooms_per_area_raw.groupby('Lower layer Super Output Areas Code')['Observation'].sum()
total_weighted_rooms_per_area = average_rooms_per_area_raw.groupby('Lower layer Super Output Areas Code')['weighted_rooms'].sum()
average_rooms_per_area = (total_weighted_rooms_per_area / total_houses_per_area).reset_index()
average_rooms_per_area.columns = ['LSOA_code', 'housing']
print(average_rooms_per_area)
average_rooms_per_area.to_csv("housing.csv", index=False)

average_distance_per_area_raw = pd.read_csv(fr"{folder}\input\census\TS058-Distance-Travelled-To-Work-2021-lsoa-ONS.csv")
distance_means = {
    "Less than 2km": 1,
    "2km to less than 5km": 3.5,
    "5km to less than 10km": 7.5,
    "10km to less than 20km": 15,
    "20km to less than 30km": 25,
    "30km to less than 40km": 35,
    "40km to less than 60km": 50,
    "60km and over": 70,
    "Works mainly from home": 0,
}
average_distance_per_area_raw['mean_distance'] = average_distance_per_area_raw['Distance travelled to work (11 categories)'].map(distance_means)
excluded_categories = ["Does not apply", "Works mainly at an offshore installation, in no fixed place, or outside the UK"]
average_distance_per_area_raw = average_distance_per_area_raw[~average_distance_per_area_raw['Distance travelled to work (11 categories)'].isin(excluded_categories)]
average_distance_per_area_raw['weighted_distance'] = average_distance_per_area_raw['mean_distance'] * average_distance_per_area_raw['Observation']
total_observations_per_area = average_distance_per_area_raw.groupby('Lower Layer Super Output Areas Code')['Observation'].sum()
total_weighted_distance_per_area = average_distance_per_area_raw.groupby('Lower Layer Super Output Areas Code')['weighted_distance'].sum()
average_distance_per_area = (total_weighted_distance_per_area / total_observations_per_area).reset_index()
average_distance_per_area.columns = ['LSOA_code', 'commuting']
print(average_distance_per_area)
average_distance_per_area.to_csv("commuting.csv", index=False)

employment_ratio_raw = pd.read_csv(fr"{folder}\input\census\TS066-Economic-Activity-Status-2021-lsoa-ONS.csv")
employment_ratio_raw['is_in_employment'] = employment_ratio_raw['Economic activity status (20 categories)'].str.contains('In employment').astype(int)
employment_ratio_raw['employed_count'] = employment_ratio_raw['is_in_employment'] * employment_ratio_raw['Observation']
agg_employment_ratio_raw = employment_ratio_raw.groupby('Lower Layer Super Output Areas Code').agg({
    'employed_count': 'sum',
    'Observation': 'sum'
}).reset_index()
agg_employment_ratio_raw['employment_ratio'] = agg_employment_ratio_raw['employed_count'] / agg_employment_ratio_raw['Observation']
result_employment_ratio_raw = agg_employment_ratio_raw[['Lower Layer Super Output Areas Code', 'employment_ratio']]
result_employment_ratio_raw.columns = ['LSOA_code', 'work']
print(result_employment_ratio_raw)
result_employment_ratio_raw.to_csv("work.csv", index=False)

files = os.listdir()
selected_files = [file for file in files if not any(char.isdigit() for char in file) and file.endswith('.csv')]
dfs = []
for file in selected_files:
    df = pd.read_csv(file)
    dfs.append(df)
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = pd.merge(merged_df, df, on="LSOA_code", how="outer")
merged_df.to_csv(fr"{folder}\input\censusdata.csv", index=False)

arcpy.env.workspace = folder

def cluster(folder):
    convenience_area = fr"{folder}\output.gdb\convenience_area"
    censusdata_csv = fr"{folder}\input\censusdata.csv"
    arcpy.management.CreateFileGDB(folder, "others.gdb")

    # Process: Feature Class To Feature Class (Feature Class To Feature Class) (conversion)
    analysisunit = arcpy.conversion.FeatureClassToFeatureClass(
        in_features=convenience_area, 
        out_path=fr"{folder}\others.gdb", 
        out_name="analysisunit"
    )[0]
    # Process: Join Field (Join Field) (management)
    analysisunit_2_ = arcpy.management.JoinField(in_data=analysisunit, in_field="LSOA21CD", join_table=censusdata_csv, join_field="LSOA_code")[0]
    # Process: Spatially Constrained Multivariate Clustering (Spatially Constrained Multivariate Clustering) (stats)
    analysisunit_SCMC = fr"{folder}\output.gdb\analysisunit_SCMC"
    arcpy.stats.SpatiallyConstrainedMultivariateClustering(in_features=analysisunit_2_, output_features=analysisunit_SCMC, analysis_fields=["age", "car", "commuting", "ethnic", "household", "housing", "mobility", "nonlocal", "religion", "work"], number_of_clusters=20, spatial_constraints="CONTIGUITY_EDGES_CORNERS", number_of_permutations=200, output_table="")

### run
with arcpy.EnvManager(outputCoordinateSystem="PROJCS[\"British_National_Grid\",GEOGCS[\"GCS_OSGB_1936\",DATUM[\"D_OSGB_1936\",SPHEROID[\"Airy_1830\",6377563.396,299.3249646]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"False_Easting\",400000.0],PARAMETER[\"False_Northing\",-100000.0],PARAMETER[\"Central_Meridian\",-2.0],PARAMETER[\"Scale_Factor\",0.9996012717],PARAMETER[\"Latitude_Of_Origin\",49.0],UNIT[\"Meter\",1.0]]", scratchWorkspace="C:\\Users\\xianl\\Documents\\ArcGIS\\Projects\\CASA0010\\CASA0010.gdb", workspace="C:\\Users\\xianl\\Documents\\ArcGIS\\Projects\\CASA0010\\CASA0010.gdb"):
    cluster(folder)
    arcpy.SetParameter(1, True)
    arcpy.AddMessage("Success!")

In progress for the auto weighted calculation.